
Proteus Tutorials![triton](http://proteustoolkit.org/_static/corps_logo_cross2.svg)
=================

ERDC Fluid-Structure Interaction R&D Workshop

Nov 15-16, 2017

Outline for this morning
========================

- Introduction to Proteus and basic tutorials (Chris)
- Simulating waves: `proteus.WaveTools` and wave/structure tutorials, *Aggelos Dimakopoulos*, HR Wallingford
- Simulating structures: `proteus.Geometry` and floating structure tutorials, *Tristan de Lataillade*, HR Wallingford/IDCORE
- Using Proteus for fast prototyping and numerical methods research, *Alistair Bentley*, CHL/Clemson University
- Proteus roadmap (Chris)

Current and Former Contributors
===============================
- Chris Kees, Matthew Farthing, Aron Ahmadia, CHL
- Aggelos Dimakopolous, Tristan de Lataillade, HR Wallingford 
- Steve Mattis and Tim Povich, Univeristy of Texas
- Ido Akkerman, UCSD
- Spencer Patty, Texas A&M
- Alistair Bentley,John Chrispell, and Lea Jenkins, Clemson

What is  Proteus?
=================

A Python toolkit for solving equations

Find $u:\Omega \rightarrow \mathbb{R}$ satisfying
$$
\nabla \cdot (-a\nabla u) + r(x) = 0
$$
on 
$$
\Omega = [0,1] \times [0,1]
$$
subject to
$$
u|_{\partial \Omega} = 0
$$

Import some tools

In [5]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from proteus.iproteus import *
import proteus.default_p as physics 
import proteus.default_n as numerics
from proteus.TransportCoefficients import PoissonEquationCoefficients

Define the equation

In [6]:
def r(x):
    return -2*x[...,0]*(1.0-x[...,0])-2*x[...,1]*(1.0-x[...,1])
def a(x):
    return np.array([[1.0,0.0],
                     [0.0,1.0]])
physics.coefficients = PoissonEquationCoefficients([a],[r]) 
physics.name="PoissonDots"

Define the domain and boundary conditions

In [8]:
physics.nd = 2
#physics.domain =  Domain.RectangularDomain(L=[1.0,1.0],
#                                           x=[0.0,0.0])

from proteus.mprans import SpatialTools as st
domain = Domain.PlanarStraightLineGraphDomain()

bt = {'bottom': 1,
     'right': 2,
     'top': 3,
     'left':4,
     'vv':5}
shape2 = st.CustomShape(domain,
                       vertices=[[0.0,0.0],#vertex 0
          [1.0,0.0],#1
          [1.0,1.0],#2
          [0.0,1.0]#3
        ],
vertexFlags=[5,5,5,5],
segments    =[[0,1],#segment 0
              [1,2],#1
              [2,3],#2
              [3,0]#3
             ],
segmentFlags=[ 1   , 
              2  ,
              3    ,
              4],
regions=[[0.5,0.5]],
regionFlags=[1],
boundaryTags=bt)


domain.MeshOptions.setElementSize(0.1)
domain.MeshOptions.setTriangleOptions()
domain.MeshOptions.setOutputFiles('mesh')

st.assembleDomain(domain)
physics.domain = domain

shape2.BC['left'].setFixedNodes()
shape2.BC['top'].setFixedNodes()
shape2.BC['right'].setFixedNodes()
shape2.BC['bottom'].setFixedNodes()

def getDBC(x,flag):
    if flag in [physics.domain.boundaryTags['left'],
                physics.domain.boundaryTags['top'],
                physics.domain.boundaryTags['right'],
                physics.domain.boundaryTags['bottom']]:
        return lambda x,t: 0.0
        
#getDBC = lambda x,flag:None
physics.dirichletConditions = {0: lambda x, flag: physics.domain.bc[flag].hx_dirichlet.init_cython()}
# def getDFBC(x, flag):
#         #if x[0] == 0:
#     if flag in [physics.domain.boundaryTags['left'],
#                 physics.domain.boundaryTags['top'],
#                 physics.domain.boundaryTags['right'],
#                 physics.domain.boundaryTags['bottom']]:
#         return lambda x,t: 0.0
    
#physics.diffusiveFluxBoundaryConditions = {0:{0:getDFBC}}

Configure the numerics

In [9]:
numerics=default_n
numerics.femSpaces = {0:FemTools.P1}
numerics.elementQuadrature = Quadrature.SimplexGaussQuadrature(physics.nd,3)
numerics.elementBoundaryQuadrature = Quadrature.SimplexGaussQuadrature(physics.nd-1,3)
# numerics.nn = 10
numerics.he=0.1
numerics.triangleOptions = domain.MeshOptions.triangleOptions
numerics.numericalFluxType = NumericalFlux.Diffusion_SIPG_exterior
so = default_so
so.name = physics.name
so.sList=[default_s]
so.tnList = [0,1]

Solve the problem

In [10]:
ns = NumericalSolution.NS_base(so,[physics],[numerics],so.sList,opts)
failed = ns.calculateSolution('run1')
assert(not failed)

AttributeError: 'mprans.BoundaryConditions.BC_RANS' object has no attribute 'he_dirichlet'

Plot the solution

In [11]:
model = ns.modelList[0].levelModelList[-1]
plt.figure()
plt.tricontourf(model.mesh.nodeArray[:,0],
                model.mesh.nodeArray[:,1],
                model.mesh.elementNodesArray,
                model.u[0].dof)

import matplotlib.tri as tri
# Create the Triangulation; no triangles so Delaunay triangulation created.
triang = tri.Triangulation(model.mesh.nodeArray[:,0],
                          model.mesh.nodeArray[:,1],
                          model.mesh.elementNodesArray)
plt.figure()
plt.gca().set_aspect('equal')
plt.triplot(triang, 'b.-', lw=0.1)

NameError: name 'ns' is not defined

A Python toolkit for research on ***continuum models***

In [ ]:
class NonlinearPoisson(TransportCoefficients.TC_base):
    def __init__(self,A,rofx):
        TransportCoefficients.TC_base.__init__(self, 
                         nc=1, 
                         variableNames=['u'],
                         diffusion = {0:{0:{0:'constant'}}},
                         potential = {0:{0:'nonlinear'}},
                         reaction  = {0:{0:'constant'}})
        self.A=A;
        self.rofx=rofx
    def evaluate(self,t,c):
        c[('a',0,0)][...,0] = self.A[0][0]
        c[('a',0,0)][...,3] = self.A[1][1]
        c[('r',0)][:]       = self.rofx(c['x'][:])
        c[('phi',0)][:]     = c[('u',0)]**2+10.0*c[('u',0)]
        c[('dphi',0,0)][:]  = 2.0*c[('u',0)] + 10.0

In [ ]:
physics.coefficients = NonlinearPoisson([[1.0,0.0],[0.0,1.0]], r)
numerics.tolFac=0.0
numerics.nl_atol_res = 1.0e-8
#numerics.nn=21

In [ ]:
ns = NumericalSolution.NS_base(so,[physics],[numerics],so.sList,opts)
failed = ns.calculateSolution('run2')
assert(not failed)

In [ ]:
model = ns.modelList[0].levelModelList[-1]
plt.figure()
plt.tricontourf(model.mesh.nodeArray[:,0],
                model.mesh.nodeArray[:,1],
                model.mesh.elementNodesArray,
                model.u[0].dof)

A Python toolkit for research on ***numerical methods***

In [ ]:
physics.coefficients = PoissonEquationCoefficients([a],[r]) 
numerics.femSpaces={0:FemTools.P2}
ns = NumericalSolution.NS_base(so,[physics],[numerics],so.sList,opts)
failed = ns.calculateSolution('run3')
assert(not failed)

In [ ]:
model = ns.modelList[0].levelModelList[-1]
import matplotlib.tri as tri
triang = tri.Triangulation(model.u[0].femSpace.dofMap.lagrangeNodesArray[:,0],
                model.u[0].femSpace.dofMap.lagrangeNodesArray[:,1])

plt.figure()
plt.tricontourf(triang, model.u[0].dof)

What does this have to do with FSI?
===================================

- Many open questions or choices still exist about models and methods
- Many more open questions existed ten years ago
- There is no universal FSI formulation
- There is no best method for all flow regimes and applications

***To succeed in modeling FSI we need a platform for conducting research and streamlining the path from research to production models.*** 

![cartoon](images/fsi_simple.png)

Brief description of formulations and methods
=============================================
- Hybrid level set/volume-of-fluid methods for topological change
- Signed distance solvers (Eikonal equation)
- Transport solvers (VOF, Level Set)
- Navier-Stokes solvers (DNS, LES, RANS)
- Moving mesh methods for some solid/fluid interfaces
- Immersed/embedded methods for other solid/fluid interfaces

<video controls src="images/ubbink_vel.mp4" />

<video controls src="images/wigleyFineSurfZ.mp4" />

<video controls src="images/dtmb.mp4" />

<video controls alt="test" src="images/vofOpSlow4.mp4"/>


Air-Water Verification and Validation
=====================================
- Cooperative project with HR  Wallingford
- https://github.com/erdc-cm/air-water-vv